In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tqdm.auto import tqdm

sns.set_style('darkgrid')

# Dataset Exploration

In [ ]:
#đọc dữ liệu
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')

#kích thước của mảng
print(train.shape)

#In mảng
train.head()

Ta có thể thấy được ảnh "1" mang nhiều hơn 1 nhãn nên đây là dạng bài đa nhãn nên ở đây chúng ta sử dụng 
Multi-label classification cho bài toán

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

#sử dụng split() để tách nhãn trong label
#sử dụng MultiLabelBinarizer() tạo ma trận theo các tag đã tách được dưới dạng cột
mlb = MultiLabelBinarizer().fit(train.labels.apply(lambda x: x.split()))
labels = pd.DataFrame(mlb.transform(train.labels.apply(lambda x: x.split())), columns=mlb.classes_)

#khởi tạo figure và axes
fig, ax = plt.subplots(figsize=(20, 6))

#In biểu đồ dưới dạng cột
labels.sum().plot.bar(title='Target Class Distribution');

In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))

#In biểu đồ số lượng tag có trong tiều đề của mỗi bức ảnh (vd: 1tag, 2tag, 3tag)
labels.sum(axis=1).value_counts().plot.bar(title='Distribution of Number of Labels per Image');

In [ ]:
#khởi tọa 1 figure có 3x4 axes
fig, ax = plt.subplots(3, 4, figsize=(20, 10))

#gán từng ảnh thứ 2 của mảng trong từng loại label vào từng vị trí axes từ trái sang phải và từ trên xuống dưới
for i, img in enumerate(train.groupby('labels').first().reset_index().values):
    ax[i//4][i%4].imshow(plt.imread(f"../input/plant-pathology-2021-fgvc8/train_images/{img[1]}"))
    ax[i//4][i%4].set_title(img[0])
    ax[i//4][i%4].axis('off')
fig.suptitle('Image Samples', fontsize=18); 

# Preprocessing and Augmentation

In [ ]:
#sử dụng pd.concat() để ghép nối image với labels
labels = pd.concat([train['image'], labels], axis=1)
labels.head()

 Bây giờ nếu chúng ta sử dụng ngay ảnh nói trên để train cho model CNN Classify thì sẽ bị hiện tượng Overfit vì dữ liệu nhiều nhưng đa phần giống nhau. Dẫn đến train sẽ có chất lượng tốt nhưng khi test sẽ thấy không nhận chuẩn lắm.
 
 Chúng ta sẽ thực hiện augment dữ liệu để làm phong phú hơn dữ liệu, tăng data variance , tăng tính tổng quát cho model 
bằng ImageDataGenerator của Keras.

In [ ]:
# scale pixel values to [0, 1]
# validation_split dùng 90% dữ liệu đầu tiên để đào tạo và 10% dữ liệu tiếp theo để kiểm tra
image_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.1)

# flow_from_dataframe để đọc ảnh từ mảng đã tạo ở trên
# x_col = tên hình ảnh
# y_col = tên nhãn
# class_mode = raw: đề nghị trình tạo trả về tất cả các giá trị trong y.
# target_size=(224, 224): thay đổi kích thước hình ảnh thành 224 x 224 pixel.
# batch_size=64: là mỗi lần cập nhật trọng số, ta dùng 64 images.
train_generator = image_data_generator.flow_from_dataframe(
    dataframe=labels,
    directory='../input/plant-pathology-2021-fgvc8/train_images',
    x_col='image',
    y_col=labels.columns.tolist()[1:],
    class_mode='raw',
    color_mode="rgb",
    target_size=(224, 224),
    batch_size=64,
    subset='training'
)

valid_generator = image_data_generator.flow_from_dataframe(
    dataframe=labels,
    directory='../input/plant-pathology-2021-fgvc8/train_images',
    x_col='image',
    y_col=labels.columns.tolist()[1:],
    class_mode='raw',
    color_mode="rgb",
    target_size=(224, 224),
    batch_size=64,
    subset='validation'
)

# Modelling

Tại bước này, mình sẽ sử dụng baseModel đó là mạng MobileNetV2

In [ ]:
inputs = tf.keras.Input(shape=(224, 224, 3))

# Load MobileNetV2
x = tf.keras.applications.MobileNetV2(include_top=False)(inputs)
# Load GlobalAveragePooling2D()
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# tạo các layer mới với Dense() với '6' là số class và sử dụng hàm sigmoid
# Lấy x từ output của GlobalAveragePooling2D()
outputs = tf.keras.layers.Dense(6, activation='sigmoid')(x)

# Tạo model với output là lớp Dense vừa thêm
model = tf.keras.models.Model(inputs, outputs)
# Compile model
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=1e-4))

# In cấu trúc mạng
model.summary()
tf.keras.utils.plot_model(model, show_shapes=True)

Tổng số tham số của mạng là 2,265,670 trong đó có 2,231,558 đã fix cứng, còn lại 34,112 tham số sẽ được train.

In [ ]:
rlp = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.01)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)

history = model.fit(train_generator, validation_data=valid_generator, epochs=10, callbacks=[rlp, es])

In [ ]:
fix, ax = plt.subplots(figsize=(20, 6))
pd.DataFrame(history.history)[['loss', 'val_loss']].plot(ax=ax, title='Model Loss Curve')

# Submission

In [ ]:
submissions = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
submissions.head()

In [ ]:
test_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_data_generator.flow_from_dataframe(
    submissions,
    directory = '../input/plant-pathology-2021-fgvc8/test_images',
    x_col="image",
    y_col=None,
    target_size=(224, 224),
    color_mode="rgb",
    classes=None,
    class_mode=None,
    shuffle=False,
    batch_size=1
)

predictions = model.predict(test_generator,steps=len(test_generator.filenames))

In [ ]:
thresh = 0.5
for i in range(3):
    submissions.iloc[i, 1] = ' '.join(labels.columns[1:][predictions[i] >= thresh])
    
submissions.to_csv('submission.csv', index=False)    

In [ ]:
model.save('mobilenetv2.h5')